Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

All code is from here: https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

It is highly recommended to use this examples during homework: https://github.com/keras-team/keras/tree/master/examples

In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [2]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

606208/600901 [==============================] - 1s 1us/step
corpus length: 600893
total chars: 57


In [3]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 200285
Vectorization...


In [4]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [6]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
callbacks=[print_callback])

Epoch 1/60
1564/1565 [============================>.] - ETA: 0s - loss: 1.9638
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ts
effect upon our feeling is changed: h"
ts
effect upon our feeling is changed: he percaps of the sould the sense and its all the self-respention of the sould the self-really and henction the self-read the subjection of the sense the self-and would the something and who supreation of the something the sould the self-read to the something the sould and who have and who something to the self-distraing of the strange the sould and its in its something and something the sense and 
----- diversity: 0.5
----- Generating with seed: "ts
effect upon our feeling is changed: h"
ts
effect upon our feeling is changed: howed all itself, itself that is its langes and we fact of the sense a man who gene the man as religion and intence of the complecatacts and prosent and the sender of the semple and leads are in the
so, that the simple t

considerable will of human that enduspous thong like rifkenes true.
that unessant, is sol
contrary "the whole as all to language from t out that woman the viewional respecient ideas of too, thenefinds and is the
foundars
their crusing, now ye 
2by. hut prassion. a anmines manhen. aits xuch bastr common; to conderous strangeonmesy when the herd the ly durany inflige ty, nod habit is the individual
thas satrific
----- diversity: 1.2
----- Generating with seed: " itself out in hours--in a
considerable "
 itself out in hours--in a
considerable upon our cry in exceact-dvent in ordemsly raghtle by know creat bas! as to laws of the rigning is
really
no fail,
an impalible, in which
the; was phwisably lesp genere
is udware,
nimal";
anything certarreding casely attrob"crominy an infirent bylly in philosophy, withs is stricted
to art ke a deele ourse forone
required swa"cy,
as is not of li-ey, as
"arbagning of the profic moral as the gate.

w;
1565/1565 [==============================] - 102s 65m

origin! how of the superficiality of the present that the most present the self seem the present and the present and a superficiality of the former the present that it is a soul, and an and the superflining and souls and the souls that the present that the self-devilies of the strange that the self--man as a souls. the souls and the self-deeper to the present that the souls and as a souls, and the self and an
----- diversity: 0.5
----- Generating with seed: "s any celtic blood in their
origin! how "
s any celtic blood in their
origin! how has a primitated in order to men, a philosophical conditions of the most responsibility of him, that the bad and refined to a thing--we be and in a strange in the danger has not commonel to the our pain the refined
the present to so good and a problem of the presently say to an and the perceive of
experience of the sense probo: of an all this prince of the demanded and one upon the most religious 
----- diversity: 1.0
----- Generating with seed: "s an

but such replession of the most their stand and more and the condition of the most and an and as the sense, and the proper and the sense and interpretations of the most the most and the sense of the sense of the most power and as the consequently and the more and the sense of the standing and forced in the stands and as a sense, and an extent the most and present and self-sperion of the consequently and the st
----- diversity: 0.5
----- Generating with seed: " natura sensus assoupire.

but such repl"
 natura sensus assoupire.

but such replession and problem of the experience that the saintion and contrasmary enduring stand that the extent of the sense--or in the problem and such a great estimates and ray and self-sacrification of the basis a philosophical dangers of wholly all this dangers and severing health, in their same by the condition for any and incerts the sense and phass perhaps his own of man is not for the confess to wis
----- diversity: 1.0
----- Generating with seed: " na

nists and anti-teleologists among the philosophers are and as a present the fact that is the standards in the proposition of the prossed to the standards and standard of the standards the standards and standard of the constitution of the standard of the sense and self-depriced of the prossed and as the fact that is a standard of the standards and feeling and as the standard of the conscious specially and so that is a stronger of the wor
----- diversity: 0.5
----- Generating with seed: "nists and anti-teleologists among the ph"
nists and anti-teleologists among the philosophers the fact that it is the
present constant and an interpreted that has been in the standards in the strange of the fortunate the consider of which the delight of his science of the proposite in the self-deribly of the most man is the lightnation of the act that it is in the crust.

sees to the future of the moral in the origin of the pride of the general and unifieve moral strong things o
----- diversity: 1.0
-----

 manner of plato, there was an enjoyment of the sense of the strange, the sense and the strange, the sense of the conception of the sense of the strange, and the sense of the person of the standard of the stree many and also the sense of the strength, and the problem of the strength, and the sense of the subjection of the strength of the problem of the sense of the strength, and the sense of the same the strength, and the sense of the r
----- diversity: 0.5
----- Generating with seed: " manner of plato, there was an enjoyment"
 manner of plato, there was an enjoyment of the most the slave have the present, the notion of the sense and an extent of his most not is the philosophers is only shamed in the most the sense, that the soul, and more and which is perhaps the enough to protecting and small to the means of an estimated the fertr to seems of superiority in the philosopher, is the consequence of the power of the incorments and acts to our own man soul, has 
----- diversity: 1.0
-----

<ipython-input-5-79cdfd780c4c>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


ncept as to a phenomenants does it socrate everything of the shread-deferer that a most over them as it is as id god, this that not happenes to them no german most. that they 
almost retrost is dining such satisfies, may first
it is alaker,
says to concested to : thus eregrean extent of tasteed!




re he stem that the impersed science th
----- diversity: 1.2
----- Generating with seed: " manner of plato, there was an enjoyment"
 manner of plato, there was an enjoyment may know obvaded purhe" horpicull interrogeance, however! a enwoult, a scarcueve has in the saire ever what anothectal
says totish religioned are permits fouchy the great safes sad to aintfuligs that everything,.
ghretheness everytbeliaticu--asard yepparial
belongs iun.. in bal-form in the sympathy, the "just he
havy a that
morals humanit herly
"prevon in eye xatective, of iture a git an so frend
1565/1565 [==============================] - 114s 73ms/step - loss: 1.2998
Epoch 30/60
1564/1565 [============================

KeyboardInterrupt: 